In [ ]:
from pathlib import Path
from os import getcwd, mkdir
import random
import math
import copy
import numpy as np
import matplotlib.pyplot as plt
from glycogen_module.core import GlycogenStructure
from glycogen_module.algorithm import gillespie_step

random.seed(123)

In [ ]:
SCRIPTDIR = current_path = getcwd()
PARAMS_PATH =  Path(f"{SCRIPTDIR}/../testdata/parameters_tab1.json")


In [ ]:
OUT_PATH =  Path(f"{SCRIPTDIR}/../testdata/tmp_data_for_densities")
try:
    mkdir(OUT_PATH)
except FileExistsError as e:
    print(e)

num_glucose_to_be_fixed = 3000
num_simulations = 5
snapshot_interval = 500

size_spec_gbe_leftover = 3
size_spec_gbe_transferred = 3
size_spec_gbe_spacing = 1


In [ ]:
for i in range(num_simulations):
    g = GlycogenStructure.from_json_file(
        PARAMS_PATH)
    assert g.gs == 0.2
    assert g.gbe == 1.0
    assert g.gp == 0.0
    assert g.gde == 0.0
    assert g.l_gbe_leftover == size_spec_gbe_leftover
    assert g.l_gbe_transferred == size_spec_gbe_transferred
    assert g.l_gbe_spacing == size_spec_gbe_spacing
   
    N = g.get_num_glucose_fixed()
    while N <= num_glucose_to_be_fixed:
        reaction, time = gillespie_step(g)
        if reaction == 'Act_gs()':
            try:
                g.act_gs()
            except Exception as e:
                print('act_gs:', e)
        elif reaction == 'Act_gp()' and N > 10:
            try:
                g.act_gp()
            except Exception as e:
                print('act_gp:',e)
        elif reaction == 'Act_gbe()':
            try:
                g.act_gbe_flexible_model()
            except Exception as e:
                print('act_gbe:',e)
        elif reaction == 'Act_gde()':
            try:
                g.act_gde()
            except Exception as e:
                print('act_gde:',e)

        N = g.get_num_glucose_fixed()
        if N % snapshot_interval == 0:
            g.export_to_file(Path(f"{OUT_PATH}/g{N}_{i}.json"))

In [ ]:
filenames  = [f"g{snapshot_interval*(k+1)}_{i}.json" for k in range(int(num_glucose_to_be_fixed/snapshot_interval)) for i in range(num_simulations)]
total_occupancy = []

# Averaging over simulations
for s in range(0, num_glucose_to_be_fixed+1, snapshot_interval):
    if s == 0:
        continue
    occupancy_list = []
    for i in range(num_simulations):
        filename = f"{OUT_PATH}/g{s}_{i}.json"
        from_snapshot = GlycogenStructure.from_json_file(filename, no_init=True)
        occupancy_list.append(from_snapshot.get_density_profile())
    total_occupancy.append(np.mean(occupancy_list, axis=0))

In [ ]:
cmap = plt.cm.Blues
colorbis = cmap(np.linspace(0.15, 0.85, len(total_occupancy)))
plt.figure(figsize=(10, 8))
for i in range(len(total_occupancy)):
    plt.plot(0.24*np.linspace(1, 100, 50),
             total_occupancy[i], color=colorbis[i])
plt.xlabel('radius [nm] ', fontsize='30', labelpad=20)
plt.ylabel('occupancy', fontsize='30', labelpad=20)
plt.tick_params(axis='both', width=3, labelsize=25)
plt.ylim([0, 0.35])
plt.show()